In [9]:
import pandas as pd
import re
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_scheduler
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv("labeled_data.csv")
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
data = df[['tweet', 'class']]
#verify unique target values
data['class'].unique()

array([2, 1, 0])

In [7]:
def preprocess(text):
    text = re.sub(r"http\S+", "", text)  #remove URLs
    text = re.sub(r"[^a-zA-Z]", " ", text)  #remove special characters and numbers
    text = text.lower()  #convert to lowercase
    words = text.split()
    return " ".join(words)

data['clean_text'] = data['tweet'].apply(preprocess)

C:\Users\17987\AppData\Local\Temp\ipykernel_33208\1316877197.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['clean_text'] = data['tweet'].apply(preprocess)


In [11]:
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize the text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Return as dictionary
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def train_epoch(model, dataloader, optimizer, scheduler, device):
    """Train the model for one epoch"""
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(dataloader, desc="Training")
    for batch in progress_bar:
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        # Update progress bar
        progress_bar.set_postfix({'loss': loss.item()})
    
    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    """Evaluate the model on validation data"""
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            # Get predictions
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    report = classification_report(true_labels, predictions, target_names=['Hate Speech', 'Offensive', 'Neither'], output_dict=True)
    conf_matrix = confusion_matrix(true_labels, predictions)
    
    return report, conf_matrix, predictions, true_labels

# def plot_confusion_matrix(conf_matrix, class_names):
#     """Plot confusion matrix as heatmap"""
#     plt.figure(figsize=(10, 8))
#     sns.heatmap(
#         conf_matrix, 
#         annot=True, 
#         fmt='d', 
#         cmap='Blues', 
#         xticklabels=class_names,
#         yticklabels=class_names
#     )
#     plt.xlabel('Predicted')
#     plt.ylabel('True')
#     plt.title('Confusion Matrix')
#     plt.tight_layout()
#     plt.savefig('confusion_matrix.png')
#     plt.close()

def train_and_evaluate(df, model_name="distilbert-base-uncased", epochs=3, batch_size=16, learning_rate=5e-5):
    """Train and evaluate the model"""
    # Split data into train, validation, and test sets
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])
    train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['class'])
    
    print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")
    
    # Load pre-trained tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, 
        num_labels=3,
        id2label={0: "Hate Speech", 1: "Offensive", 2: "Neither"},
        label2id={"Hate Speech": 0, "Offensive": 1, "Neither": 2}
    )
    
    # Create datasets
    train_dataset = HateSpeechDataset(train_df['clean_text'].values, train_df['class'].values, tokenizer)
    val_dataset = HateSpeechDataset(val_df['clean_text'].values, val_df['class'].values, tokenizer)
    test_dataset = HateSpeechDataset(test_df['clean_text'].values, test_df['class'].values, tokenizer)
    
    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
    
    # Check if GPU is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model.to(device)
    
    # Set up optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = epochs * len(train_dataloader)
    scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )
    
    # Train the model
    print(f"Starting training for {epochs} epochs...")
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, device)
        print(f"Training loss: {train_loss:.4f}")
        
        # Evaluate on validation set
        val_report, val_matrix, _, _ = evaluate(model, val_dataloader, device)
        print(f"Validation F1-Score (weighted): {val_report['weighted avg']['f1-score']:.4f}")
    
    # Final evaluation on test set
    print("\nFinal evaluation on test set:")
    test_report, test_matrix, test_preds, test_labels = evaluate(model, test_dataloader, device)
    
    # Print results
    print("\nClassification Report:")
    print(pd.DataFrame(test_report).T)
    
    # Plot confusion matrix
    #plot_confusion_matrix(test_matrix, ['Hate Speech', 'Offensive', 'Neither'])
    
    # Save the model
    model.save_pretrained("hate_speech_classifier")
    tokenizer.save_pretrained("hate_speech_classifier")
    print("Model saved to 'hate_speech_classifier' directory")
    
    return model, tokenizer, test_report

def predict_text(text, model, tokenizer, device):
    """Make prediction for a single text"""
    model.eval()
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    # Move to device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        pred = torch.argmax(outputs.logits, dim=1).item()
    
    labels = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    return labels[pred]


# Train and evaluate model
model, tokenizer, test_report = train_and_evaluate(
    data, 
    model_name="distilbert-base-uncased",  # You can change to other models
    epochs=3,
    batch_size=16,
    learning_rate=5e-5
)

# Example predictions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

examples = [
    "I love everyone regardless of their background",
    "You're an idiot and nobody likes you",
    "I hate people from that country, they should all die"
]

print("\nExample predictions:")
for example in examples:
    prediction = predict_text(example, model, tokenizer, device)
    print(f"Text: '{example}'")
    print(f"Prediction: {prediction}\n")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train: 17843, Validation: 1983, Test: 4957
Using device: cuda
Starting training for 3 epochs...

Epoch 1/3


Training: 100%|██████████| 1116/1116 [03:49<00:00,  4.87it/s, loss=1.06]  


Training loss: 0.2933


Evaluating: 100%|██████████| 124/124 [00:09<00:00, 12.74it/s]


...

Training loss: 0.1424


Evaluating: 100%|██████████| 124/124 [00:03<00:00, 37.09it/s]


Validation F1-Score (weighted): 0.9100

Final evaluation on test set:


Evaluating: 100%|██████████| 310/310 [00:08<00:00, 38.36it/s]



Classification Report:
              precision    recall  f1-score     support
Hate Speech    0.545872  0.416084  0.472222   286.00000
Offensive      0.942784  0.961699  0.952148  3838.00000
Neither        0.908981  0.899160  0.904043   833.00000
accuracy       0.919710  0.919710  0.919710     0.91971
macro avg      0.799212  0.758981  0.776138  4957.00000
weighted avg   0.914203  0.919710  0.916374  4957.00000
Model saved to 'hate_speech_classifier' directory

Example predictions:
Text: 'I love everyone regardless of their background'
Prediction: Neither

Text: 'You're an idiot and nobody likes you'
Prediction: Neither

Text: 'I hate people from that country, they should all die'
Prediction: Hate Speech



In [ ]:
text = "example"
predict_text(text, model, tokenizer, device)

'Offensive Language'